In [ ]:
import os

import pandas as pd
import numpy as np
import math, os, random, json, pickle
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from copy import deepcopy

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem.Fingerprints import FingerprintMols

import torch
import shutil

import os
import csv
from tqdm import tqdm
import shutil

from madrigal.utils import DATA_DIR, BASE_DIR

from torchdrug import data, utils
from torchdrug.core import Registry as R

In [ ]:
# Download data
path = '/path/to/Madrigal/modality_pretraining/str/pretraining/molecule-datasets/'
url = "http://deepchem.io.s3-website-us-west-1.amazonaws.com/datasets/muv.csv.gz"
md5 = "9c40bd41310991efd40f4d4868fa3ddf"
file_name = utils.download(url, path, md5=md5)

In [ ]:
def remove_similar(df, name, root):
    
    smiles = df.smiles.tolist()
    
    test_between = pd.read_csv(DATA_DIR+'polypharmacy_new/DrugBank/split_by_drugs_random/test_between_df.csv')
    test_within = pd.read_csv(DATA_DIR+'polypharmacy_new/DrugBank/split_by_drugs_random/test_within_df.csv')
    all_smiles = pd.read_pickle(DATA_DIR+'polypharmacy_new/combined_metadata_ddi.pkl')
    all_smiles = all_smiles.canonical_smiles
    test = np.concatenate((test_between['head'].values, test_within['head'].values, test_within['tail'].values ))
    test = np.unique(test)
    test_smiles = all_smiles.loc[test].values
    
    def get_fingerprints(l):
        all_mols = [AllChem.AddHs(Chem.MolFromSmiles(sm)) for sm in tqdm(l)] 
        all_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=256) for mol in tqdm(all_mols)]
        return all_fps

    fps = get_fingerprints(smiles)
    test_fps = get_fingerprints(test_smiles)
    
    scores = []
    for n in tqdm(fps):
        score = DataStructs.BulkTanimotoSimilarity(n, test_fps)
        scores.append(score)
        
    scores = np.array(scores)
    msk = np.where(scores >= 0.98)
    to_remove = msk[0]
    
    for i, smile in enumerate(smiles):
        try:
            x = Chem.MolFromSmiles(smile)
        except:
            print(f'Removed {i}')
    
    bad_df = df.index.isin(to_remove)
    df = df[~bad_df]
    
    df.to_csv(os.path.join(root, f'{name}_filtered_torchdrug_new.csv'), index=False)

In [ ]:
df = pd.read_csv(file_name, header=None)
remove_similar(df, 'muv', path)